<a href="https://colab.research.google.com/github/seina001/QLC240_Spring2025/blob/main/Exercise11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 9: Importing and initial visualization of data that we will be modeling

The following exercises will be organized into 4 sections:
    1. Comparing the 2 datasets (JMA, Argo) and looking at overlaps and gaps.
    2. Creating models to find the best fit model for the JMA dataset, to use on the ARGO data.
    3. Use only data from JMA that corresponds to the geographic area of the ARGO dataset. Then do the same thing as in section 2.
    4. Analysis of the best fit models from sections 2 and 3, to see if using data from the same geographic area results in a better model.

In [2]:
#Import packages needed to run code
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

Import the JMA dataset and visualize to see what we have.
IMport nitrate file as a pandas dataset.

In [3]:
infile = 'jma_nitrate.csv'
#######

#End the cell by visualizing the data variable, if that is what you called it, example:
data

NameError: name 'data' is not defined

Plotting a scater plot of nitrate and depth data from JMA cruise.
You might want to invert the axis, to make it look like an ocean water column.

In [ ]:
# Scatter plot of Nitrate vs Depth for JMA data
plt.figure(figsize=(8, 6))

plt.scatter(jma_df["nitrate"], jma_df["depth"], alpha=0.5, color="blue")

# Invert the y-axis to resemble an ocean water column
plt.gca().invert_yaxis()

# Labels and title
plt.xlabel("Nitrate (µM)")
plt.ylabel("Depth (m)")
plt.title("Nitrate vs Depth (JMA Cruise)")

plt.show()

The raw jma data appear to have some nitrate values at -999, representing either no data or bad data. These have to be removed before we can continue.

In [ ]:
#Example:
# Looks like there are -999 values to indicate no samples or bad samples. Let's take those out.
corr_data = data[data.nitrate>0]
#Plot the corrected data in a scatter plot again.

The following cell will read in another file, and fix some date/time variables so they are managable. You don't need to change it at all, except you need to upload the .csv file itself.

In [ ]:
# Reading in the Argo float data with no NO3 values
infile = 'kuro_BGCargo_noNO3.csv'
data_BGC = pd.read_csv(infile, sep = (','))

# Separating the date column of BGC argo data into year month and day columns to look at seasonality compared to the jma data.
import datetime

data_BGC['year'] = pd.DatetimeIndex(data_BGC['Date/GMT']).year
data_BGC['month'] = pd.DatetimeIndex(data_BGC['Date/GMT']).month
data_BGC['day'] = pd.DatetimeIndex(data_BGC['Date/GMT']).day
#data_BGC.head()

# Section 1, compare datasets (JMA, ARGO)

Compare the samples locations and see how the datasets overlap or don't overlap

In [ ]:
#Plot the corrected data and the Argo float data "Latitude" and "Longitude" coordinates. If you're unsure of the column to choose, just look at the data and the column head names.
#Scatter plot for example

In [4]:
# Compare geographic coordinates
print("\nJMA Longitude:", jma_df["lon"].min(), "-", jma_df["lon"].max())
print("Argo Longitude:", argo_df["LON"].min(), "-", argo_df["LON"].max())

print("\nJMA Latitude:", jma_df["lat"].min(), "-", jma_df["lat"].max())
print("Argo Latitude:", argo_df["LAT"].min(), "-", argo_df["LAT"].max())

NameError: name 'jma_df' is not defined

Just from the geographic extend, we can see that the distribution of JMA samples is considerably higher than that of BCG ARGO samples. However big this despair, for this and other variables, there is some overlap in the sampled region, depths, and seasons. The geographical extent overlaps in the region between the ~30 to ~40° of latitude and ~150 to ~175° of longitude.

In [ ]:
# Compare  depth ranges
print("\nJMA Depth Range:", jma_df["depth"].min(), "-", jma_df["depth"].max())
print("Argo Depth Range:", argo_df["DEPTH[M]"].min(), "-", argo_df["DEPTH[M]"].max())


Compare the water masses that were sampled in both datasets

In [ ]:
#Plot a scatter plot looking at the temperature vs depth profile, and seperate plot looking at salinity vs temperature. Each data source should be labeled differently.

In [ ]:
plt.figure(figsize=(8, 6))

plt.scatter(jma_df["sal"], jma_df["theta"], alpha=0.5, label="JMA", color="blue")
plt.scatter(argo_df["SALINITY[PSS]"], argo_df["TEMPERATURE[C]"], alpha=0.5, label="Argo", color="red")

plt.xlabel("Salinity (PSS)")
plt.ylabel("Temperature (°C)")
plt.title("Temperature-Salinity Diagram (JMA vs Argo)")
plt.legend()
plt.show()

# Section 2, model testing and best fit analysis using entire dataset
Start of model testing, and comparison to find the best fit model for this dataset using KNN.
We chose to use temperature and salinity as variables to model nitrate from. They are good proxies for water masses and can hopefully be used to predict nitrate.

In [5]:
#import the jma nitrate file, then get columns I want for training
infile = 'jma_nitrate.csv'
nitratefile = pd.read_csv(infile,sep=(','))
nitratefilefiltered = nitratefile[nitratefile.nitrate>0]
X = nitratefilefiltered[["sal","theta"]]
y = nitratefilefiltered[["nitrate"]]
print(np.shape(X))
print(np.shape(y))


#split data set to train/test 20/80 split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)



(15574, 2)
(15574, 1)


In [10]:
# generate the data
import matplotlib.pyplot as plt
import random


# pick model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

#run KNN with K neighbors ranging from 1-15, calculating
#r2 and RMSE to find the best fit model
results = []
for n in range(1,16,1):
    res = []
    model = KNeighborsRegressor(n_neighbors=n)
    model.fit(X_train, y_train)
    #print ('Neighbors = %s\n' % (int(n)))
    # compute the R^2 score
    accuracyscore = model.score(X_train,y_train)
    #print("Training R^2 score: {0:.2f}\n".format(model.score(X_train,y_train)))
    # plot the model together with the data
    xfit = X_test
    yfit = model.predict(xfit)
    #print("Testing R^2 score: {0:.2f}\n".format(model.score(xfit,yfit)))
    rmse = sqrt(mean_squared_error(y_test, yfit))
    #print("RMSE = %s \n" % (rmse))
    res = [n,accuracyscore,rmse]
    results.append(res)
results

[[1, 0.9999110613989356, 2.6058678198832435],
 [2, 0.9927289125889681, 2.4303135395420776],
 [3, 0.989436962483429, 2.320717656355632],
 [4, 0.9877493952444761, 2.294790385486795],
 [5, 0.9862445125445002, 2.265924014995549],
 [6, 0.9853651009567571, 2.2178185162292237],
 [7, 0.9846615070565908, 2.213675727092634],
 [8, 0.9840785865559012, 2.1952765454848047],
 [9, 0.9836823893549954, 2.20225936338761],
 [10, 0.9833161917430909, 2.1939401652725508],
 [11, 0.9830143132143836, 2.2023368473610616],
 [12, 0.98254256174158, 2.2144196381599355],
 [13, 0.9820371954017075, 2.2041353557622165],
 [14, 0.9816750176585883, 2.208859357144669],
 [15, 0.9812961480610605, 2.2152382383662066]]

In [11]:
myarray = np.asarray(results)
myarray

array([[ 1.        ,  0.99991106,  2.60586782],
       [ 2.        ,  0.99272891,  2.43031354],
       [ 3.        ,  0.98943696,  2.32071766],
       [ 4.        ,  0.9877494 ,  2.29479039],
       [ 5.        ,  0.98624451,  2.26592401],
       [ 6.        ,  0.9853651 ,  2.21781852],
       [ 7.        ,  0.98466151,  2.21367573],
       [ 8.        ,  0.98407859,  2.19527655],
       [ 9.        ,  0.98368239,  2.20225936],
       [10.        ,  0.98331619,  2.19394017],
       [11.        ,  0.98301431,  2.20233685],
       [12.        ,  0.98254256,  2.21441964],
       [13.        ,  0.9820372 ,  2.20413536],
       [14.        ,  0.98167502,  2.20885936],
       [15.        ,  0.98129615,  2.21523824]])

In [12]:
#Make list into array to plot nicely, in order to look at all of the
#results

myarray = np.asarray(results)

#make a dataframe with each column named for ploting
data = pd.DataFrame({'Kneighbors': myarray[:,0], 'AccScore': myarray[:,1], 'RMSE': myarray[:,2]})
data


,Kneighbors,AccScore,RMSE
0,1.0,0.999911,2.605868
1,2.0,0.992729,2.430314
2,3.0,0.989437,2.320718
3,4.0,0.987749,2.294790
4,5.0,0.986245,2.265924
5,6.0,0.985365,2.217819
6,7.0,0.984662,2.213676
7,8.0,0.984079,2.195277
8,9.0,0.983682,2.202259
9,10.0,0.983316,2.193940


Looking at the table above we see that the best fit model is using 10 K neighbors. The accuracy is high and RMSE is lowest.

In [13]:
#Run best fit model, and display results
model = KNeighborsRegressor(n_neighbors=10)
model.fit(X_train, y_train)
print ('Neighbors = %s\n' % (10))
    # compute the R^2 score
accuracyscore = model.score(X_train,y_train)
print("Training R^2 score: {0:.2f}\n".format(model.score(X_train,y_train)))
    # plot the model together with the data
xfit = X_test
yfit = model.predict(xfit)
print("Testing R^2 score: {0:.2f}\n".format(model.score(xfit,yfit)))
rmse = sqrt(mean_squared_error(y_test, yfit))
print("RMSE = %s \n" % (rmse))



Neighbors = 10

Training R^2 score: 0.98

Testing R^2 score: 1.00

RMSE = 2.1939401652725508 



The next two plots show the predicted nitrate vs salinity (and then temperature). Both training and test datasets are plotted. We can see here that the predicted variables do align with the general test data sets and their aren't many outliers. Meaning there doesn't seem to be overtraining on any specific data points.

In [ ]:
#Plot salinity vs. nitrate predicted
plt.scatter(X_train[["sal"]],y_train)
plt.scatter(xfit[["sal"]], yfit,c = 'k')
plt.xlabel('Salinity')
plt.ylabel('Nitrate (uM)')
plt.title('Nitrate vs Salinity, Test and Training data sets')
plt.legend(('Training dataset', 'Test dataset'), edgecolor = 'k')
#Plot temperature vs. nitrate predicted
plt.scatter(X_train[["theta"]],y_train)
plt.scatter(xfit[["theta"]], yfit,c = 'k')
plt.xlabel('Temperature (C)')
plt.ylabel('Nitrate (uM)')
plt.title('Nitrate vs Temperature, Test and Training data sets')
plt.legend(('Training dataset', 'Test dataset'), edgecolor = 'k')
#Plot predicted nitrate from test compared to actual nitrate from test
#this is using the best fit model decided above
from sklearn.linear_model import LinearRegression
plt.scatter(yfit,y_test,color = 'y')
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(yfit, y_test)  # perform linear regression
Y_pred = linear_regressor.predict(yfit)  # make predictions
plt.plot(yfit, Y_pred, color='red')
plt.xlabel("Predicted nitrate variables using the test dataset")
plt.ylabel("Actual nitrate variables from the test dataset")
plt.title("KNN nitrate predict with regression (n = 10)")
plt.show()
r2 = r2_score(yfit, Y_pred)
print("r2 = %s \n" % (r2))
rmse = sqrt(mean_squared_error(yfit, Y_pred))
print("RMSE = %s \n" % (rmse))
The plot and r2/RMSE values above show that the predicted nitrate values for the test dataset compare well to the actual nitrate values. There are some outliers seen on the graph but the statistics show that the model is a great fit overall on the test dataset.